# Learning from AirBnB Data

Starting with Tom Lee's CSV from : https://s3.amazonaws.com/tomslee-airbnb-data-2/washington.zip, Let's see if we can answer a few questions :

1. Are the listing ID's unique - that is, the dataset has one entry (row) per room ID?
2. In this dataset, what is the maximum number of listings that any host has?
3. Can we predict the price given some features of the listing (an exercise in K-Nearest-Neighbors supervised machine learning and hyperparameters (N) tuning (optimization) and feature selection - find out which ones really help you build a good model)?

The learning that enabled this came from DataQuest's online course..

In [2]:
# Step one, read in the CSV and get a sense of the dataframe - do you need to do any cleaning etc (KNN is not forgiving)
import pandas as pd
dc_listings = pd.read_csv("tomslee_airbnb_washington_1433_2017-07-11.csv")  # did you know you could type
                                  # a few letters of the name and hit TAB and Jupyter will fill it in?
dc_listings.head(2)

,room_id,survey_id,host_id,room_type,country,city,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,minstay,last_modified,latitude,longitude,location
0,3732219,1433,280636,Shared room,NaN,Washington,NaN,Columbia Heights,0,0.0,3,1.0,NaN,129.0,NaN,2017-07-11 08:53:56.381540,38.931081,-77.030618,0101000020E6100000D02A33A5F54153C0A77686A92D77...
1,15087225,1433,90860645,Shared room,NaN,Washington,NaN,Brentwood,6,5.0,4,1.0,NaN,118.0,NaN,2017-07-11 08:53:55.616987,38.908054,-77.003306,0101000020E61000005B785E2A364053C041800C1D3B74...


As a novice data-science practitioner, some questions arise rightaway :

- What if I want to use the room_id as the index for each row? Is that possible? (Yes)
- What if the price, which, in these two rows, looks very well behaved actually had commas and $ characters? Is it possible to automatically convert such "almost truly numeric" columns into truly numeric ones automatically? (A fact of data-science life that, to their credit, DataQuest does drill into you, is that work involves a lot of drudgery - data cleaning)
- Is it possible to (again, automatically) reduce the dataframe to just the numeric columns that really lend themselves to some kind of predictive analysis?

For a start, let's make sure that room_id does have as many unique values as the length of the df :

In [4]:
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8237 entries, 0 to 8236
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   room_id               8237 non-null   int64  
 1   survey_id             8237 non-null   int64  
 2   host_id               8237 non-null   int64  
 3   room_type             8237 non-null   object 
 4   country               0 non-null      float64
 5   city                  8237 non-null   object 
 6   borough               0 non-null      float64
 7   neighborhood          8237 non-null   object 
 8   reviews               8237 non-null   int64  
 9   overall_satisfaction  8237 non-null   float64
 10  accommodates          8237 non-null   int64  
 11  bedrooms              8237 non-null   float64
 12  bathrooms             0 non-null      float64
 13  price                 8237 non-null   float64
 14  minstay               0 non-null      float64
 15  last_modified        

Clearly, this dataset has been given some attention already - because the columns seem to be all-or-nothing. The real life - especially where one is using data that is available-but-not-advertised, is likely to require much more work.

First off, can we set_index to room_id? How many unique room_id's are there?

In [5]:
len( dc_listings['room_id'].unique() )

8235

Clearly, that's a bit depressing :) We were hoping for 8237. What happened?

In [9]:
dc_listings['room_id'].value_counts().head()

10862880    2
11528631    2
16123903    1
15115590    1
6171801     1
Name: room_id, dtype: int64

In [13]:
dc_listings[dc_listings['room_id'] == 10862880]

,room_id,survey_id,host_id,room_type,country,city,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,minstay,last_modified,latitude,longitude,location
7278,10862880,1433,11827226,Private room,NaN,Washington,NaN,Stadium-Armory,47,5.0,3,1.0,NaN,75.0,NaN,2017-07-11 01:37:36.439387,38.902561,-76.981948,0101000020E6100000D9976C3CD83E53C091D26C1E8773...
7279,10862880,1433,11827226,Private room,NaN,Washington,NaN,Capitol Hill,47,5.0,3,1.0,NaN,75.0,NaN,2017-07-11 01:37:36.439387,38.902561,-76.981948,0101000020E6100000D9976C3CD83E53C091D26C1E8773...


Okay, going by latitude/longitude, and other things also being equal, suggests that this was just an error in data prep. How does one drop duplicate rows (keep only one)? The other option, since we have plenty of data is to simply drop rows whose room_id is not unique. How?
    As it turns out this is not an uncommon problem, so the developers of pandas have made `drop_duplicates` available with **keep** {‘_first_’, ‘_last_’, `False`}, default ‘_first_’. So..


In [16]:
dc_listings.drop_duplicates( subset='room_id', inplace=True )
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8235 entries, 0 to 8236
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   room_id               8235 non-null   int64  
 1   survey_id             8235 non-null   int64  
 2   host_id               8235 non-null   int64  
 3   room_type             8235 non-null   object 
 4   country               0 non-null      float64
 5   city                  8235 non-null   object 
 6   borough               0 non-null      float64
 7   neighborhood          8235 non-null   object 
 8   reviews               8235 non-null   int64  
 9   overall_satisfaction  8235 non-null   float64
 10  accommodates          8235 non-null   int64  
 11  bedrooms              8235 non-null   float64
 12  bathrooms             0 non-null      float64
 13  price                 8235 non-null   float64
 14  minstay               0 non-null      float64
 15  last_modified        

In [18]:
dc_listings.set_index('room_id', inplace=True)
dc_listings.head()

,survey_id,host_id,room_type,country,city,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,minstay,last_modified,latitude,longitude,location
room_id,,,,,,,,,,,,,,,,,,
3732219,1433,280636,Shared room,NaN,Washington,NaN,Columbia Heights,0,0.0,3,1.0,NaN,129.0,NaN,2017-07-11 08:53:56.381540,38.931081,-77.030618,0101000020E6100000D02A33A5F54153C0A77686A92D77...
15087225,1433,90860645,Shared room,NaN,Washington,NaN,Brentwood,6,5.0,4,1.0,NaN,118.0,NaN,2017-07-11 08:53:55.616987,38.908054,-77.003306,0101000020E61000005B785E2A364053C041800C1D3B74...
19634784,1433,138150306,Shared room,NaN,Washington,NaN,South West,1,0.0,4,1.0,NaN,84.0,NaN,2017-07-11 08:53:53.434225,38.884121,-77.019518,0101000020E6100000BCEB6CC83F4153C0795A7EE02A71...
18547685,1433,26180779,Shared room,NaN,Washington,NaN,Shaw,11,5.0,2,1.0,NaN,74.0,NaN,2017-07-11 08:53:49.654605,38.910593,-77.023461,0101000020E6100000D0EE9062804153C0B77BB94F8E74...
13878076,1433,2387207,Shared room,NaN,Washington,NaN,Cleveland Park,2,0.0,2,1.0,NaN,50.0,NaN,2017-07-11 08:53:48.721169,38.935485,-77.059807,0101000020E61000009A44BDE0D34353C00473F4F8BD77...


Okay, the second question about wealthy hosts should be easy, right? :)

In [21]:
dc_listings['host_id'].value_counts().head()

30283594    109
25188        90
46630199     49
8160186      39
8008783      38
Name: host_id, dtype: int64

In [33]:
dc_listings[ dc_listings['host_id'] == 30283594 ]['bedrooms'].value_counts()

1.0    67
2.0    41
0.0     1
Name: bedrooms, dtype: int64

In [34]:
dc_listings[ dc_listings['host_id'] == 30283594 ]['price'].describe()

count    109.000000
mean     190.596330
std       41.477598
min      129.000000
25%      169.000000
50%      181.000000
75%      199.000000
max      349.000000
Name: price, dtype: float64

Nice, so this rich (person or group?) has 109 places, one of them can't claim to even have a bedroom, but can still command $129 per night (assuming that is the minimum :) ). What does this tell you about DC? :)

In [36]:
dc_listings[ dc_listings['host_id'] == 30283594 ]['overall_satisfaction'].value_counts()

0.0    102
5.0      3
4.5      2
3.5      1
4.0      1
Name: overall_satisfaction, dtype: int64

Assuming that a 0 satisfaction score means "not reviewed yet" (else, the min would be 1, right?), this tells us that these people have a lot of new listings waiting to be rented.

In [35]:
dc_listings['overall_satisfaction'].value_counts()

0.0    3473
5.0    3135
4.5    1422
4.0     168
3.5      29
3.0       5
2.5       2
2.0       1
Name: overall_satisfaction, dtype: int64

The other thing I'm wondering about is what is the highest "accommodates" to "bedrooms" ratio? You'd think that two per room would be reasonable. But, given what we know about DC already..

In [37]:
(dc_listings['accommodates']/dc_listings['bedrooms'] ).describe()

count    8235.000000
mean             inf
std              NaN
min         0.333333
25%         2.000000
50%         2.500000
75%         4.000000
max              inf
dtype: float64

Okay, a good lesson - know your data - we've seen already that #bedrooms can be 0, so, we obviously should only use the data that has #bedrooms > 0..

In [38]:
dc_lists_wbedrooms = dc_listings[ dc_listings['bedrooms'] > 0 ]
(dc_lists_wbedrooms['accommodates']/dc_lists_wbedrooms['bedrooms'] ).describe()

count    7441.000000
mean        2.557789
std         1.024299
min         0.333333
25%         2.000000
50%         2.000000
75%         3.000000
max        16.000000
dtype: float64

Okay, it would be nice to figure out the mystery of these outliers. Wouldn't it? :)

In [46]:
(dc_lists_wbedrooms['accommodates']/dc_lists_wbedrooms['bedrooms'] ).value_counts().sort_index(ascending=False).head(10)

16.000000       3
8.000000        7
7.000000        1
6.000000       84
5.000000      123
4.666667        2
4.500000        5
4.333333        2
4.000000     1090
3.750000        4
dtype: int64

We'd like to know the bedrooms and accommodates values for these guys wouldn't we? Unfortunately, I don't think it can be done without adding a column.. I'm not that good.. :)

In [48]:
dc_lists_wbedrooms.loc["guests_per"] = dc_lists_wbedrooms['accommodates']/dc_lists_wbedrooms['bedrooms']
dc_lists_wbedrooms[ dc_lists_wbedrooms['guests_per'] == 16]

C:\Users\anant\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,survey_id,host_id,room_type,country,city,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,minstay,last_modified,latitude,longitude,location,guests_per
room_id,,,,,,,,,,,,,,,,,,,
11574642,1433.0,27353.0,Shared room,NaN,Washington,NaN,Anacostia,25.0,4.0,16.0,1.0,NaN,125.0,NaN,2017-07-11 08:53:41.453278,38.876157,-76.963657,0101000020E6100000EFE3688EAC3D53C0AB949EE92570...,16.0
10416451,1433.0,27353.0,Shared room,NaN,Washington,NaN,Anacostia,29.0,4.0,16.0,1.0,NaN,15.0,NaN,2017-07-11 08:53:24.207717,38.876158,-76.962800,0101000020E6100000AF25E4839E3D53C07B1002F22570...,16.0
15984634,1433.0,27353.0,Shared room,NaN,Washington,NaN,Anacostia,23.0,4.0,16.0,1.0,NaN,11.0,NaN,2017-07-11 08:53:22.703522,38.876514,-76.964267,0101000020E61000002C67EF8CB63D53C03A3E5A9C3170...,16.0


What does such a place look like? Here's an example (note, dataset is from 2017, but this picture is from today (Nov,2020) :

<img src="shared_room_DC16.PNG"/>


Ok, now that we're done playing :), let's see about price prediction. As you might expect, the right way is to use modules that already exist - SciKit-Learn to be precise..

Since we only have `reviews`, `overall_satisfaction`, `accommodates` and `bedrooms` to work with, we will loop over inclusions of all of these and then sweep the number of nearest neighbours.

The model we end up with is finally judged on the Mean Squared Error.

In [49]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [54]:
N_split = int( len( dc_listings )*0.8 )   # 80% for training, rest for test
train_df = dc_listings[0:N_split]
test_df = dc_listings[N_split:]

In [62]:
features_to_try = ['reviews', 'overall_satisfaction', 'bedrooms']  # assuming that accommodates is not negotiable.. 
hyper_params = list( range( 5,25) )   # try 2,3...13 for number of nearest neighbours
features = ['accommodates']
mse = None
for feature in features_to_try :
    features.append( feature )
    for param in hyper_params :
        knn = KNeighborsRegressor( n_neighbors=param, algorithm='brute')
        knn.fit( train_df[ features], train_df['price'])
        predictions = knn.predict( test_df[features])
        new_mse = mean_squared_error( test_df['price'], predictions )
#         print( "Info : with {} MSE = {}".format( param, new_mse))
        if mse :
            if new_mse  < mse :
                mse = new_mse
                opt_N = param
        else :
            mse = new_mse   # how to DRY here?
            opt_N = param
    print( "With {} for features, optimal N is {} for MSE of {}".format( features, opt_N, mse))
    mse = None


With ['accommodates', 'reviews'] for features, optimal N is 19 for MSE of 5002.192353426948
With ['accommodates', 'reviews', 'overall_satisfaction'] for features, optimal N is 20 for MSE of 4813.2695901639345
With ['accommodates', 'reviews', 'overall_satisfaction', 'bedrooms'] for features, optimal N is 6 for MSE of 5575.447868177831


Let's just see what we get with just accommodates..

In [63]:
hyper_params = list( range( 5,25) )   # try 2,3...13 for number of nearest neighbours
features = ['accommodates']
mse = None
for param in hyper_params :
    knn = KNeighborsRegressor( n_neighbors=param, algorithm='brute')
    knn.fit( train_df[ features], train_df['price'])
    predictions = knn.predict( test_df[features])
    new_mse = mean_squared_error( test_df['price'], predictions )
    if mse :
        if new_mse  < mse :
            mse = new_mse
            opt_N = param
    else :
        mse = new_mse   # how to DRY here?
        opt_N = param
print( "With {} for features, optimal N is {} for MSE of {}".format( features, opt_N, mse))


With ['accommodates'] for features, optimal N is 14 for MSE of 6462.838797814206


Good news then :) We do get better with reviews included (number of reviews. Makes sense doesn't it - the more people that have stayed at a listing, the more value it has)

Here's a good data-science question : How much do bad reviews hurt? That is, if you exclude overall_satisfaction and then look at nearest neighbours, how much below the predicted value are you? And, how is that correlated with the overall_satisfaction?

Unfortunately, given that our error is so high (on the order of $70 std.deviation) we might not get anywhere.. So what can one do? Actually, given that we only have an aggregate number for satisfaction, we can't answer this one. We would need all th

## Conclusion

Use **20** nearest neighbours and `accommodates`, `reviews`, and `overall_satisfaction` as the features.